# Création et peuplement de la base de données

## Imports

In [2]:
import pprint
import pymongo
import json
from utils.utils import build_key
from tqdm import tqdm
from difflib import SequenceMatcher, get_close_matches

## Connexion

In [3]:
client = pymongo.MongoClient()

## Sources de données

In [4]:
FILEPATH_VOIE = "streets/odp/data/voie.json"
FILEPATH_HIST = "streets/odp/data/noms_voies_actuelles_paris.json"
FILEPATH_APUR = "streets/apur/APUR_quartiers.geojson"
FILEPATH_WKPD = "streets/wkpd/out/wkpd_streets.json"


## Voies de Paris

In [3]:

with open(FILEPATH_VOIE) as f :
    data_voie = json.load(f)

collection_voie = client["visitp"]["streets_voies"]
collection_voie.insert_many(data_voie)


## Historique des voies

In [ ]:
with open(FILEPATH_HIST) as f :
    data_hist = json.load(f)

collection_hist = client["visitp"]["streets_hist"]
collection_hist.insert_many(data_hist)    

## APUR : les quartiers de Paris

In [ ]:
with open(FILEPATH_APUR) as f :
    quartiers = json.load(f)

collection_quartiers = client["visitp"]["streets_quartiers"]
collection_quartiers.insert_many(quartiers["features"])

# GeoSON Polygon : description du contour du quartier
collection_quartiers.create_index([("geometry", pymongo.GEOSPHERE)])

## Données Wikipedia

In [5]:
with open(FILEPATH_WKPD) as f :
    wkpd_streets = json.load(f)

collection_wkpd = client["visitp"]["wkpd_streets"]
collection_wkpd.insert_many(wkpd_streets)

## Relations entre les collections

### Insertion d'un champ historique dans `collection_voies` 

Cette étape réalise la liaison entre la base principale `voie.json` et la base `noms_voies_actuelles_paris.json` qui contient un historique des voies parisiennes. Cette liaison est réalisée en insérant, dans la base principale, une clé `hist_recordid` qui pointe vers l'enregistrement correspondant dans la base `noms_voies_actuelles_paris.json`.

In [6]:
for doc in tqdm(collection_voie.find(), total = collection_voie.count()):
    c_voie_vp = doc["fields"]["c_voie_vp"]
    # pprint.pprint(c_voie_vp)
    q = collection_hist.find_one({"fields.cvoie": c_voie_vp})
    # print(q["recordid"])
    if q : collection_voie.update_one( {"_id": doc["_id"]}, { "$set": { "hist_recordid" : q["recordid"]}})

100%|██████████| 6513/6513 [00:40<00:00, 162.34it/s]


### Insertion d'une clé dans `collection_voies`

Pour lier les données Wikipedia, on crée, dans la base principale, une clé forgée de la même manière que la clé de la base Wikipedia. Le nom de la rue est transformé par la fonction `build_key()`.

In [6]:
# f = open("log.txt", mode='w')
collection_voie = client["visitp"]["streets_voies"]
for doc in tqdm(collection_voie.find(), total = collection_voie.count()):
    l_longmin = doc["fields"]["l_longmin"]
    key = build_key(l_longmin)
    # f.write(key)
    collection_voie.update_one( {"_id": doc["_id"]}, { "$set": { "key" : key}} )
    # f.write("\t\t\t\t\t\tOK\n")
print("OK")

    

100%|██████████| 6075/6075 [00:03<00:00, 1526.64it/s]OK



## Création des index de recherche

Des index doivent être créés pour autoriser la recherche dans la base MongoDB.

In [9]:

# GeoSON Point : point milieu de la voie
collection_voie.create_index([("geometry.coordinates", pymongo.GEOSPHERE)])
collection_voie.create_index([("geometry.coordinates", pymongo.GEO2D)])

# GeoSON LineString : description de la géométrie de la voie
collection_voie.create_index([("fields.geom", pymongo.GEOSPHERE)])

# Index pour la recherche textuelle
# collection_voie.create_index([("fields.l_longmin", pymongo.TEXT)])
collection_voie.create_index( [ ("fields.l_longmin", pymongo.TEXT),
                                ("key", pymongo.TEXT)  ])



'fields.l_longmin_text_key_text'

## Insertion d'un champ Wikipedia dans collection_voies

Sur la base de la clé créée ci dessus, pour chaque voie de la base principale, on recherche la correspondance exacte dans la base Wikipedia.

### Recherche d'une correspondance exacte et insertion en cas de succès

In [17]:
collection_voie = client["visitp"]["streets_voies"]
collection_wkpd = client["visitp"]["wkpd_streets"]
found = []
not_found = []
for doc in tqdm(collection_voie.find(), total = collection_voie.count()):
    voie_key = doc["key"]
    q = collection_wkpd.find_one( {"key" : voie_key})
    # print(q)
    # if q: found.append(q)
    if q :
        found.append(q)
        collection_voie.update_one( {"_id": doc["_id"]}, { "$set": { "wkpd" : q["href"]}})
        collection_voie.update_one( {"_id": doc["_id"]}, { "$set": { "arr" : q["arr"]}})
    else: not_found.append(voie_key)


100%|██████████| 6075/6075 [00:30<00:00, 201.22it/s]


In [18]:
print("found :", len(found), "not found :", len(not_found))

found : 5706 not found : 369


5706 clés de `collection_voies` ont été trouvées dans `collection_wkpd`. Pour chacune de ces clés un champ `wkpd` et un champ `arr` ont été insérés. On crée une liste des clés Wikipedia pour faire une recherche par similarité pour celles qui n'ont pas été trouvées.

### Recherche de similarité pour les clés non trouvées

Pour la recherche de similarité, on se base sur la fonction `get_close_matches()` du module [difflib](https://docs.python.org/3/library/difflib.html).

In [6]:
# Liste des clés uniques dans la base Wikipedia

wkpd_keys = collection_wkpd.distinct("key")
len(wkpd_keys)

5898

In [10]:
similarities = []
for key in not_found:
    d = dict()
    d["key"] = key
    d["match"] = None
    sims = get_close_matches(key, wkpd_keys)
    d["sims"] = sims
    # print(key)
    similarities.append( d )

In [12]:
len(similarities)

369

La liste de similarités (sur les 369 clés qui n'ont pas de stricte équivalence dans la base Wikipedia) est inscrite dans le fichier `similarities_before.json`.

In [13]:
with open("similarites_before.json", mode='w') as f:
    json.dump(similarities, f)

Le fichier `similarities_before.json` est parsé à la main, la clé `match` modifiée si une similarité a été détectée et enregistré sous `similarities_after.json`.

### Insertion pour les similarités détectées manuellement

In [22]:
with open("similarities_after.json", mode='r') as f:
    sims = json.load(f)

In [23]:
len(sims)

369

In [41]:
# Détection des similarités détectées manuellement

valids = dict()
for sim in sims:
    if sim["match"] is not None: 
        valids[sim["key"]] = sim['sims'][int(sim["match"])]

valids

{'placedefontenoyunesco': 'placedefontenoy',
 'placemarieclaudevaillantcouturier': 'placemarieclaudevaillantcouturieretpierrevillon',
 'ruemontalembert': 'ruedemontalembert',
 'placeduhuitnovembre1942': 'placedu8novembre1942',
 'peristyledemontpensier': 'galeriedemontpensier',
 'peristyledevalois': 'galeriedevalois',
 'placedu22novembre1943': 'placeduvingtdeuxnovembre1943',
 'placebilalberenni': 'placebilalberreni',
 'placejeanpaulsartresimonedebeauvoir': 'placejeanpaulsartreetsimonedebeauvoir',
 'ruedeplhineseyrig': 'ruedelphineseyrig',
 'squarelarochefoucauld': 'squaredelarochefoucauld',
 'ruedemagenta': 'ruemagenta',
 'placeemanuellevinas': 'placeemmanuellevinas',
 'impasseduquaranteneuffaubourgsaintmartin': 'impassedu49faubourgsaintmartin',
 'placedudixhuitjuin1940': 'placedu18juin1940',
 'avenuedestilleuls12eme': 'avenuedestilleuls',
 'placeduhuitfevrier1962': 'placedu8fevrier1962',
 'ruegeorgespicquart': 'ruemariegeorgespicquart',
 'placedutrocaderoetduonzenovembre': 'placedutroc

In [40]:
for key in valids.keys():
    voie = collection_voie.find_one( {"key" : key})
    wkpd_key = valids[key]
    q = collection_wkpd.find_one( {"key" : wkpd_key})
    collection_voie.update_one( {"_id": voie["_id"]}, { "$set": { "wkpd" : q["href"]}})
    collection_voie.update_one( {"_id": voie["_id"]}, { "$set": { "arr" : q["arr"]}})

